In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

SETTING = kraft.read_json("../project.json")

if "project_directory_path" in SETTING:

    SETTING.update(
        kraft.read_json(os.path.join(SETTING["project_directory_path"], "project.json"))
    )

    SETTING["output_directory_path"] = os.path.join(
        SETTING["project_directory_path"], "output"
    )

PATH = make_path_dict(SETTING)

In [ ]:
assert SETTING["signal_type"] == "context"

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for element_type, dataframe in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    if element_type not in SETTING["context_element_types"]:

        continue

    output_tsv_file_path = PATH["{}_x_fit_parameter.tsv".format(element_type)]

    element_x_fit_parameter = kraft.fit_each_dataframe_row_to_skew_t_pdf(
        dataframe, n_job=SETTING["n_job"], output_tsv_file_path=output_tsv_file_path
    )

    element_x_fit_parameter = pd.read_csv(output_tsv_file_path, sep="\t", index_col=0)

    for parameter, parameter_values in element_x_fit_parameter.items():

        parameter_values_not_na_unique = parameter_values.dropna().unique()

        if parameter_values_not_na_unique.size == 1:

            continue

        kraft.plot_histogram((parameter_values,), layout=layout)

        n_extreme_per_direction = 2

        extreme_elements = kraft.select_series_indices(
            parameter_values,
            "<>",
            n=n_extreme_per_direction,
            layout={
                "title": {
                    "text": "Fit {}".format(SETTING["{}_alias".format(element_type)])
                },
                "xaxis": {"title": {"text": parameter}},
            },
        )

        kraft.plot_histogram(
            tuple(
                dataframe.loc[extreme_element] for extreme_element in extreme_elements
            ),
            layout={
                "title": {"text": "Extreme {}".format(parameter)},
                "xaxis": {"title": {"text": SETTING["feature_x_sample_value_name"]}},
            },
        )

        for extreme_element in extreme_elements:

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                extreme_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            kraft.plot_context(
                dataframe.loc[extreme_element],
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                layout={
                    "xaxis": {"title": {"text": SETTING["feature_x_sample_value_name"]}}
                },
            )